In [31]:
import requests
import xmltodict
import asyncio
from typing import Dict, Tuple, Union, Optional, List

def make_request(url: str, params: dict) -> requests.Response:
    response = requests.get(url=url, params=params)
    return response

async def api_request(url: str, params: dict) -> requests.Response:
    return await asyncio.to_thread(make_request, url, params)

async def main():
    client = BGGClient()
    tasks = [client.get_game_market_data(i) for i in range(1, 10)]
    result = await asyncio.gather(*tasks)
    return result

class BGGClient:
    def __init__(self, base_url: str = "https://boardgamegeek.com/xmlapi2"):
        self.base_url = base_url

    async def get_game_market_data(self, game_id: int, retry_delay: int = 5, max_retries: int = 3) -> Tuple[str, Dict[str, Union[str, float]], float]:
        """
        Fetches detailed market information for a specific game
        
        Args:
            game_id: BGG game ID
            retry_delay: Seconds to wait between retries
            max_retries: Maximum number of retry attempts
            
        Returns:
            Tuple of (game_name, min_price_data, average_price)
        
        Raises:
            Exception: If API request fails or data processing fails
        """
        endpoint = f"{self.base_url}/thing"
        params = {
            "id": game_id,
            "type": "boardgame",
            "marketplace": 1 
        }
        
        response_data = None
        for attempt in range(max_retries):
            response = await api_request(url=endpoint, params=params)
            
            if response.status_code == 200:
                response_data = xmltodict.parse(response.content)
                # print(f"Debug: Response data for game ID {game_id}: {response_data}")  # Debug logging
                break
            elif response.status_code == 202:
                print(f"Request queued, retrying in {retry_delay} seconds...")
                await asyncio.sleep(retry_delay)
                continue
            else:
                response.raise_for_status()
        
        if not response_data:
            raise Exception(f"Failed to get response after {max_retries} attempts")
            
        try:
            # Extract game name
            if isinstance(response_data['items']['item']['name'], list):
                # If 'name' is a list, take the first element's '@value'
                game_name = response_data['items']['item']['name'][0]['@value']
            else:
                # If 'name' is not a list, directly access '@value'
                game_name = response_data['items']['item']['name']['@value']

            print(game_name)
            
            # Process marketplace listings
            min_value = {'link': '', 'value': float('inf')}
            price_list = []
            
            # Handle case where there might be single or multiple listings
            listings = response_data['items']['item'].get('marketplacelistings', {}).get('listing', [])
            if not isinstance(listings, list):
                listings = [listings]
                
            for listing in listings:
                if listing.get('price', {}).get('@currency') != 'EUR':
                    continue
                price = float(listing['price']['@value'])
                price_list.append(price)
                if price < min_value['value']:
                    min_value['link'] = listing.get('link', {}).get('@href', '')
                    min_value['value'] = price

            if not price_list:
                return (game_name, {'link': 'NA', 'value': float('inf')}, 0)

            avg_price = sum(price_list) / len(price_list)
            return (game_name, min_value, avg_price)
            
        except KeyError as e:
            raise Exception(f"Failed to process game data: {str(e)}")
        except ValueError as e:
            raise Exception(f"Failed to process pricing data: {str(e)}")
        except Exception as e:
            raise Exception(f"Unexpected error processing game data: {str(e)}")

if __name__ == "__main__":
    try:
        # Check if an event loop is already running
        try:
            loop = asyncio.get_running_loop()
        except RuntimeError:  # No event loop running
            loop = None

        if loop:
            # If running in an environment with an existing event loop (e.g., Jupyter notebook)
            import nest_asyncio
            nest_asyncio.apply()  # Allow nested event loops
            result = loop.run_until_complete(main())
        else:
            # If running as a standalone script
            result = asyncio.run(main())
        
        for i in result:
            print(i)
    except Exception as e:
        print(f"Error: {str(e)}")

Cathedral
Dragonmaster
Samurai
El Caballero
Acquire
Tal der Könige
Die Macher
Mare Mediterraneum
Lords of Creation
('Die Macher', {'link': 'https://boardgamegeek.com/market/product/3636729', 'value': 18.0}, 65.62899999999999)
('Dragonmaster', {'link': 'NA', 'value': inf}, 0)
('Samurai', {'link': 'https://boardgamegeek.com/market/product/3630338', 'value': 60.0}, 79.39933333333333)
('Tal der Könige', {'link': 'https://boardgamegeek.com/market/product/672651', 'value': 19.0}, 29.181818181818183)
('Acquire', {'link': 'https://boardgamegeek.com/market/product/3605513', 'value': 3.0}, 53.52687500000001)
('Mare Mediterraneum', {'link': 'https://boardgamegeek.com/market/product/650976', 'value': 175.0}, 175.0)
('Cathedral', {'link': 'https://boardgamegeek.com/market/product/2780856', 'value': 12.0}, 30.1)
('Lords of Creation', {'link': 'https://boardgamegeek.com/market/product/3040534', 'value': 9.0}, 22.412499999999998)
('El Caballero', {'link': 'https://boardgamegeek.com/market/product/2223

In [20]:
result

NameError: name 'result' is not defined

In [5]:
print(game_details['items']['item']['marketplacelistings'])

{'listing': [{'listdate': {'@value': 'Mon, 19 Nov 2018 19:35:29 +0000'}, 'price': {'@currency': 'GBP', '@value': '20.00'}, 'condition': {'@value': 'verygood'}, 'notes': {'@value': 'U.K. Shipping &#194;&#163;3, free collection from Brighton&#10;&#10;One token missing'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1673901', '@title': 'marketlisting'}}, {'listdate': {'@value': 'Mon, 15 Apr 2019 15:16:03 +0000'}, 'price': {'@currency': 'EUR', '@value': '20.00'}, 'condition': {'@value': 'likenew'}, 'notes': {'@value': 'opened but unplayed'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1817930', '@title': 'marketlisting'}}, {'listdate': {'@value': 'Sat, 25 May 2019 09:34:47 +0000'}, 'price': {'@currency': 'GBP', '@value': '7.00'}, 'condition': {'@value': 'verygood'}, 'notes': {'@value': 'Played only once'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1852584', '@title': 'marketlisting'}}, {'listdate': {'@value': 'Fri, 31 May 2019 23:13:12 +0000

In [7]:
for i in game_details['items']['item']['marketplacelistings']['listing']:
    print(i)

{'listdate': {'@value': 'Mon, 19 Nov 2018 19:35:29 +0000'}, 'price': {'@currency': 'GBP', '@value': '20.00'}, 'condition': {'@value': 'verygood'}, 'notes': {'@value': 'U.K. Shipping &#194;&#163;3, free collection from Brighton&#10;&#10;One token missing'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1673901', '@title': 'marketlisting'}}
{'listdate': {'@value': 'Mon, 15 Apr 2019 15:16:03 +0000'}, 'price': {'@currency': 'EUR', '@value': '20.00'}, 'condition': {'@value': 'likenew'}, 'notes': {'@value': 'opened but unplayed'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1817930', '@title': 'marketlisting'}}
{'listdate': {'@value': 'Sat, 25 May 2019 09:34:47 +0000'}, 'price': {'@currency': 'GBP', '@value': '7.00'}, 'condition': {'@value': 'verygood'}, 'notes': {'@value': 'Played only once'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1852584', '@title': 'marketlisting'}}
{'listdate': {'@value': 'Fri, 31 May 2019 23:13:12 +0000'}, 'price': {'@

In [32]:
for i in game_details['items']['item']:
    print(i)

@type
@id
thumbnail
image
name
description
yearpublished
minplayers
maxplayers
poll
poll-summary
playingtime
minplaytime
maxplaytime
minage
link
marketplacelistings
